In [1]:
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
dftrain1=cd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
dftest1=cd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
#Preprocessing Training Data
dftrain1.info()

In [4]:
for i in dftrain1:
    if dftrain1[i].isnull().sum() > 100:
        dftrain1=dftrain1.drop(i,axis=1)
        dftest1=dftest1.drop(i,axis=1) #Removing those columns from test data as well
dftrain1.info()

In [5]:
dftest1.info()

In [6]:
toprocess=[]
for i in dftrain1:
    if dftrain1[i].isnull().sum() > 0:
        toprocess.append(i)
toprocess

In [7]:
inttoprocess=dftrain1[toprocess].select_dtypes(include=['float64']).columns
cattoprocess=dftrain1[toprocess].select_dtypes(include=['object']).columns

In [8]:
for i in inttoprocess:
    dftrain1[i]=dftrain1[i].fillna(dftrain1[i].mean())

In [9]:
dftrain1=dftrain1.dropna(subset=cattoprocess)
dftrain1.info()

In [10]:
#No Null values in Training Data
toprocess1=[]
for i in dftrain1:
    if dftrain1[i].isnull().sum() > 0:
        toprocess1.append(i)
toprocess1

In [11]:
#Preprocessing Test Data
dftest1.info()

In [12]:
toprocesstest=[]
for i in dftest1:
    if dftest1[i].isnull().sum() > 0:
        toprocesstest.append(i)
toprocesstest

In [13]:
inttoprocesstest=dftest1[toprocesstest].select_dtypes(include=['float64']).columns
cattoprocesstest=dftest1[toprocesstest].select_dtypes(include=['object']).columns

In [14]:
for i in inttoprocesstest:
    dftest1[i]=dftest1[i].fillna(dftest1[i].mean())

In [15]:
dftest1=dftest1.fillna(method='bfill')
dftest1.info()

In [16]:
#No Null Values now in Test Data
toprocess1test=[]
for i in dftest1:
    if dftest1[i].isnull().sum() > 0:
        toprocess1test.append(i)
toprocess1test

In [17]:
catdata=dftrain1.select_dtypes(include=['object']).columns
catdata

In [18]:
#Encoding Categorical Data in Training Data
dftrain1=cd.get_dummies(dftrain1,columns=catdata)
dftrain1

In [19]:
dftrain1.info()

In [20]:
#Encoding Categorical Values in Test Data
cattestdata=dftest1.select_dtypes(include=['object']).columns
dftest1=cd.get_dummies(dftest1,columns=cattestdata)
dftest1

In [21]:
a=dftrain1.columns.intersection(dftest1.columns)
b=dftest1.columns.intersection(dftrain1.columns)
dftrain=dftrain1[a]
dftest=dftest1[b]
dftest=dftest.drop(['Id'],axis=1)
dftrain=dftrain.drop(['Id'],axis=1)
dftrain['SalePrice']=dftrain1['SalePrice']

In [22]:
dftest.info()

In [23]:
dftrain.info()

In [24]:
#Scaling Features
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
totrain=dftrain.drop('SalePrice',axis=1)
sdftrain=ss.fit_transform(totrain)
sdftest=ss.fit_transform(dftest)

In [105]:
#Dimensionality Reduction
from cuml.decomposition import PCA
pca=PCA(n_components=100)
pca.fit(sdftrain)
x=pca.transform(sdftrain)
pca.fit(sdftrain)
xt=pca.transform(sdftest)
print(xt.shape)

In [106]:
#Model Training on df_train
y=dftrain['SalePrice']

In [107]:
from cuml import LinearRegression
lr=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'svd')
reg = lr.fit(x,y)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)

In [108]:
ypred=lr.predict(xt)
ypred

In [109]:
idd=list(range(1461,2920))
ypredlist=ypred.to_arrow().to_pylist()
data={'Id':idd,'SalePrice':ypredlist}
resdf=cd.DataFrame(data)

In [110]:
resdf

In [80]:
resdf.to_csv('Submission.csv',columns=['Id','SalePrice'],index=False)